In [25]:
import numpy as np
import pandas as pd
import matplotlib as plt
import urllib.request
from pulp import *
from IPython.display import Image, display_png
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%matplotlib inline

In [26]:
t = pd.read_csv('../csv/stochastic_matrix.csv', index_col=0).fillna(0)
t = t.T.drop([
    'C00001', # H2O
#     'C00003', # NAD
#     'C00004', # NADH
#     'C00005', # NADPH
#     'C00006', # NADP
    'C00009', # Phosphate
#     'C00010', # CoA
#     'C00011', # CO2
    'C00033', # Acetate
    'C00080', # H+
#     'C00138', # Reduced ferredoxin
#     'C00139', # Oxidized ferredoxin
    'C15602', # Quinone
    'C15603' # Hydroquinone
]).T

In [27]:
h = [
    'R00200', # Pyruvate <- Phosphoenolpyruvate
    'R00268',
    'R00341', # Oxaloacetate -> Phosphoenolpyruvate
    'R00342',
    'R00351',
    'R00361',
    'R00658',
    'R01015',
    'R01061',
    'R01070',
    'R01082',
    'R01196',
    'R01197',
    'R01325',
    'R01512',
    'R01518',
    'R01786',
    'R01899',
    'R01900',
    'R02164',
    'R02740',
    'R04779',
    'R10343'
]

In [28]:
p = t.T[h].T
for i in p:
    if sum(p[i] != 0) == 0:
        p=p.drop(i, axis=1)
        
p['C00007'] = pd.Series(np.zeros(p.shape[0]), index=p.index)
p['C00122'] = pd.Series(np.zeros(p.shape[0]), index=p.index)
p['C00186'] = pd.Series(np.zeros(p.shape[0]), index=p.index)

p = p.T.sort_index().T

for i in list(p.columns):
    url = 'http://rest.kegg.jp/get/{}'.format(i)
    savename = '../html/{}.html'.format(i)
    urllib.request.urlretrieve(url, savename)
    
g = pd.DataFrame()
for i in list(p.columns):
    g = pd.concat([g, pd.read_table('../html/{}.html'.format(i), header=None)], axis=1)
    
c = []
for i in list(g.T[1]):
    c.append(i[12:-1])
    
p = p.T

In [75]:
c

['ATP',
 'NAD+',
 'NADH',
 'NADPH',
 'NADP+',
 'Oxygen',
 'ADP',
 'CoA',
 'CO2',
 'Pyruvate',
 'Acetyl-CoA',
 '2-Oxoglutarate',
 'Oxaloacetate',
 'Succinate',
 'Phosphoenolpyruvate',
 'Succinyl-CoA',
 'Glycerone phosphate',
 'D-Glyceraldehyde 3-phosphate',
 'Fumarate',
 'Reduced ferredoxin',
 'Oxidized ferredoxin',
 '(S)-Malate',
 'Citrate',
 '(S)-Lactate',
 '3-Phospho-D-glycerate',
 '3-Phospho-D-glyceroyl phosphate',
 'alpha-D-Glucos',
 'Isocitrate',
 'cis-Aconitate',
 '2-Phospho-D-glycerate',
 'alpha-D-Glucose 6-phosphat',
 'beta-D-Fructose 6-phosphat',
 'beta-D-Fructose 1,6-bisphosphat',
 'Oxalosuccinate']

In [29]:
p['Biomass'] =  pd.Series(np.zeros(p.shape[0]), index=p.index)
p['Oxygen'] =  pd.Series(np.zeros(p.shape[0]), index=p.index)
p['Glucose'] =  pd.Series(np.zeros(p.shape[0]), index=p.index)
p['Oxophos'] = pd.Series(np.zeros(p.shape[0]), index=p.index)
p['NADP_Exchange']  = pd.Series(np.zeros(p.shape[0]), index=p.index)

# 200 alpha-D-Glucose + 10000 ATP + 10000 Acetyl-CoA + 10000 NADP <=> 10000 NADPH + 10000 ADP 
# p['Biomass']['C00002'] = -10000
# p['Biomass']['C00024'] = -10000
# p['Biomass']['C00006'] = -10000
# p['Biomass']['C00267'] = -10000
# p['Biomass']['C00008'] = 10000
# p['Biomass']['C00005'] = 10000

# Import
p['Oxygen']['C00007'] = 1
p['Glucose']['C00267'] = 1

# 2NADH + Oxygen + 5ADP <=>5ATP + 2NAD + CO2 
p['Oxophos']['C00004'] = -2
p['Oxophos']['C00007'] = -1
p['Oxophos']['C00008'] = -5
p['Oxophos']['C00002'] = 5
p['Oxophos']['C00003'] = 2
p['Oxophos']['C00011'] = 1

# NADPH + NAD+ <=> NADP+ + NADH  C00005 + C00003 <=> C00006 + C00004
p['NADP_Exchange']['C00005'] = -1
p['NADP_Exchange']['C00003'] = -1
p['NADP_Exchange']['C00006'] = 1
p['NADP_Exchange']['C00004'] = 1

In [67]:
x = []
for i in p.columns:
    if (i == 'Glucose'):
        x.append(LpVariable(i, lowBound=-100, upBound=-100, cat='Continuous'))
    
    elif (i == 'Oxygen'):
            x.append(LpVariable(i, lowBound=-100, upBound=-100, cat='Continuous'))
    
    elif (i == 'R00341') or (i == 'Oxophos'):
        # Oxaloacetate -> Phosphoenolpyruvate
        x.append(LpVariable(i, lowBound=0, upBound=10000, cat='Continuous'))
        
    elif (i == 'R00200'):
#         'R00200', # Pyruvate <- Phosphoenolpyruvate
        x.append(LpVariable(i, lowBound=-10000, upBound=0, cat='Continuous'))

    else:
        x.append(LpVariable(i, lowBound=-10000, upBound=10000, cat='Continuous'))
        
r = pd.DataFrame(x, columns=['Reaction'], index=[str(i) for i in x])

In [68]:
m = LpProblem(sense=LpMaximize)
m += r['Reaction']['Biomass']

for i in np.dot(p, x):
    m += i == 0

In [74]:
np.dot(p, x)

array([-10000.0*Biomass + 5.0*Oxophos + -1.0*R00200 + -1.0*R00341 + -1.0*R01512 + -1.0*R01786 + -1.0*R04779 + 0.0,
       -1.0*NADP_Exchange + 2.0*Oxophos + -1.0*R00342 + -1.0*R01061 + 0.0,
       1.0*NADP_Exchange + -2.0*Oxophos + 1.0*R00342 + 1.0*R01061 + 0.0,
       10000.0*Biomass + -1.0*NADP_Exchange + 1.0*R01899 + 0.0,
       -10000.0*Biomass + 1.0*NADP_Exchange + -1.0*R01899 + 0.0,
       -1.0*Oxophos + 1.0*Oxygen + 0.0,
       10000.0*Biomass + -5.0*Oxophos + 1.0*R00200 + 1.0*R00341 + 1.0*R01512 + 1.0*R01786 + 1.0*R04779 + 0.0,
       -1.0*R00351 + 1.0*R01196 + 1.0*R01197 + 0.0,
       1.0*Oxophos + 1.0*R00268 + 1.0*R00341 + -1.0*R01196 + -1.0*R01197 + 0.0,
       -1.0*R00200 + 1.0*R01196 + 0.0,
       -10000.0*Biomass + 1.0*R00351 + -1.0*R01196 + 1.0*R10343 + 0.0,
       1.0*R00268 + 1.0*R01197 + 0.0,
       -1.0*R00341 + 1.0*R00342 + 1.0*R00351 + 1.0*R00361 + 0.0,
       -1.0*R02164 + 1.0*R10343 + 0.0,
       1.0*R00200 + 1.0*R00341 + 1.0*R00658 + 0.0,
       -1.0*R01197 + -1

In [70]:
status = m.solve()
print(LpStatus[status])

Infeasible


In [71]:
y = []
for i in x:
    y.append(i.value())

f = pd.DataFrame(y, index=r.index, columns=['value'])

In [72]:
p*f.value

,R00200,R00268,R00341,R00342,R00351,R00361,R00658,R01015,R01061,R01070,...,R01900,R02164,R02740,R04779,R10343,Biomass,Oxygen,Glucose,Oxophos,NADP_Exchange
C00002,-0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-0.0,0.0,100.0,-0.0,-0.0,0.0,-0.0
C00003,0.0,0.0,0.0,-100.0,-0.0,0.0,0.0,0.0,-0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,100.0
C00004,0.0,0.0,0.0,100.0,-0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-100.0
C00005,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-100.0,-0.0,-0.0,0.0,100.0
C00006,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,100.0,-0.0,-0.0,0.0,-100.0
C00007,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.0,-100.0,-0.0,-0.0,-0.0
C00008,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-100.0,-0.0,-0.0,-0.0,-0.0
C00010,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0
C00011,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0
C00022,-0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0
